# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
orders=pd.read_csv('orders.zip')
orders.head()
orders.columns

Index(['Unnamed: 0', 'InvoiceNo', 'StockCode', 'year', 'month', 'day', 'hour',
       'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID',
       'Country', 'amount_spent'],
      dtype='object')

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# your code here

totalbyclient=orders.groupby(['CustomerID']).agg({'amount_spent':'sum'})
a=orders.groupby(['CustomerID']).agg({'amount_spent':'sum'}).quantile(0.95)
print(a)
b=orders.groupby(['CustomerID']).agg({'amount_spent':'sum'}).quantile(0.75)
print(b)

def clienttype(row,a,b):
    if row['amount_spent']>a:
        return("VIP")
    elif row['amount_spent']<b:
        return("Client")
    else:
        return("Preferred")

amount_spent    5840.182
Name: 0.95, dtype: float64
amount_spent    1661.64
Name: 0.75, dtype: float64


In [4]:
totalbyclient["client_type"]=totalbyclient.apply((lambda x: clienttype(x,5840.182,1661.64)), axis=1)

In [5]:
totalbyclient

,amount_spent,client_type
CustomerID,,
12346,77183.60,VIP
12347,4310.00,Preferred
12348,1797.24,Preferred
12349,1757.55,Preferred
12350,334.40,Client
...,...,...
18280,180.60,Client
18281,80.82,Client
18282,178.05,Client


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [28]:
# your code here
countr=orders["Country"]
mergedlist=pd.merge(left=totalbyclient,
                    right=orders,
                    how="right",
                    left_on='CustomerID',
                    right_on= 'CustomerID')

display(mergedlist)
mergedlist.groupby(["client_type"]).agg({"Country":"max"})

,CustomerID,amount_spent_x,client_type,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,Country,amount_spent_y
0,12346,77183.60,VIP,61619,541431,23166,2011,1,2,10,medium ceramic top storage jar,74215,2011-01-18 10:01:00,1.04,United Kingdom,77183.60
1,12347,4310.00,Preferred,14938,537626,85116,2010,12,2,14,black candelabra t-light holder,12,2010-12-07 14:57:00,2.10,Iceland,25.20
2,12347,4310.00,Preferred,14939,537626,22375,2010,12,2,14,airline bag vintage jet set brown,4,2010-12-07 14:57:00,4.25,Iceland,17.00
3,12347,4310.00,Preferred,14940,537626,71477,2010,12,2,14,colour glass. star t-light holder,12,2010-12-07 14:57:00,3.25,Iceland,39.00
4,12347,4310.00,Preferred,14941,537626,22492,2010,12,2,14,mini paint set vintage,36,2010-12-07 14:57:00,0.65,Iceland,23.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,18287,1837.28,Preferred,392752,570715,22419,2011,10,3,10,lipstick pen red,12,2011-10-12 10:23:00,0.42,United Kingdom,5.04
397920,18287,1837.28,Preferred,392753,570715,22866,2011,10,3,10,hand warmer scotty dog design,12,2011-10-12 10:23:00,2.10,United Kingdom,25.20
397921,18287,1837.28,Preferred,423939,573167,23264,2011,10,5,9,set of 3 wooden sleigh decorations,36,2011-10-28 09:29:00,1.25,United Kingdom,45.00
397922,18287,1837.28,Preferred,423940,573167,21824,2011,10,5,9,painted metal star with holly bells,48,2011-10-28 09:29:00,0.39,United Kingdom,18.72


,Country
client_type,
Client,Unspecified
Preferred,United Kingdom
VIP,United Kingdom


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [ ]:
# your code here

mergedlist.groupby(["client_type"]).agg({"Country":"max"})

#It is valid for both, because I don't know how to select each client type in "client_type" column